# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [4]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

Título de la Web:
Frogames
Contenido de la Web:
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprenda

['#main-content',
 '/',
 '/',
 '/users/sign_in',
 'https://cursos.frogamesformacion.com/pages/rutas',
 'https://cursos.frogamesformacion.com/pages/instructores',
 'https://cursos.frogamesformacion.com/pages/certificaciones',
 'https://cursos.frogamesformacion.com/collections',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/ingenieria-llm',
 '/courses/ingenieria-llm',
 '/courses/principios-solid',
 '/courses/principios-solid',
 '/courses/matematicas-ml-2',
 '/courses/matematicas-ml-2',
 '/courses/obsidian',
 '/courses/obsidian',
 '/courses/trigonometria-desde-cero',
 '/courses/trigonometria-desde-cero',
 '/courses/kotlin-desde-cero',
 '/courses/kotlin-desde-cero',
 '/courses/matematicas-ml',
 '/courses/matematicas-ml',
 '/courses/github-android',
 '/courses/github-android',
 '/courses/presencia-redes-sociales',
 '/courses/presencia-redes-sociales',
 '/courses/c-sharp-unity-introduccion',
 '/courses/c-sharp-unity-introduccion',
 '/courses/shade

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [5]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [6]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/bandas-sonoras-peliculas-1
/courses/bandas-sonoras-peliculas-1
/courses/ingenieria-llm
/courses/ingenieria-llm
/courses/principios-solid
/courses/principios-solid
/courses/matematicas-ml-2
/courses/matematicas-ml-2
/courses/obsidian
/courses/obsidian
/courses/trigonometria-desde-cero
/courses/trigonometria-desde-cero
/courses/kotlin-desde-cero
/courses/kotlin-desde-cero
/courses/m

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.an

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'Página Acerca de',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'Página de Investigación',
   'url': 'https://www.anthropic.com/research'},
  {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}]}

In [12]:
get_links("https://cursos.frogamesformacion.com")

{'links': [{'type': 'Pagina de Inicio',
   'url': 'https://cursos.frogamesformacion.com'},
  {'type': 'Pagina Rutas',
   'url': 'https://cursos.frogamesformacion.com/pages/rutas'},
  {'type': 'Pagina Instructores',
   'url': 'https://cursos.frogamesformacion.com/pages/instructores'},
  {'type': 'Pagina Certificaciones',
   'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'},
  {'type': 'Pagina de Nuestros Clientes',
   'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'},
  {'type': 'Pagina Frogames para Empresas',
   'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'},
  {'type': 'Pagina Premios',
   'url': 'https://cursos.frogamesformacion.com/pages/premios'},
  {'type': 'Pagina de Cursos',
   'url': 'https://cursos.frogamesformacion.com/collections'},
  {'type': 'Pagina de Afiliados',
   'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}]}

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for Enterprise
Sep 4, 2024
Alignment
·
Research
Constitutional AI: Harmlessne

In [15]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [17]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de Precios', 'url': 'https://www.anthropic.com/pricing'}]}


'Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, Wh

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}]}


# Folleto de Anthropic

## Acerca de Anthropic
Anthropic es una empresa de investigación y seguridad de inteligencia artificial con sede en San Francisco. Estamos dedicados a construir sistemas de IA que sean fiables, interpretables y controlables, trabajando para generar un impacto positivo a través de la tecnología de IA. Nuestro equipo interdisciplinario cuenta con experiencia en aprendizaje automático, física, política y desarrollo de productos, entre otros.

## Misión y Cultura
Nuestra misión es asegurar que la IA transformativa ayude a las personas y a la sociedad a prosperar. Valoramos la confianza, la colaboración y la integridad, creando un ambiente donde las decisiones se toman en conjunto y se fomenta la honestidad. 

### Nuestros Valores
1. **Misión en primer lugar**: Todos en Anthropic están orientados hacia la misión de construir IA segura y benéfica.
2. **Confianza inusualmente alta**: Fomentamos un ambiente de confianza donde se asume buena fe y se discute con respeto.
3. **Colaboración**: Creemos en el trabajo en equipo y la cooperación entre diferentes disciplinas.
4. **Simplicidad práctica**: Preferimos soluciones simples y efectivas antes de buscar complejidades innecesarias.

## Productos y Servicios
Nuestra línea de modelos de IA incluye:
- **Claude 3.5 Sonnet**: Un modelo avanzado para tareas cognitivas complejas.
- **Claude 3.5 Haiku**: El modelo más rápido, ideal para acciones ligeras.
- **Claude Opus**: Nuestro modelo más potente, adecuado para análisis complejos y tareas que requieren múltiples pasos.

### Clientes
Trabajamos con una variedad de empresas, organizaciones sin fines de lucro y grupos de la sociedad civil, apoyándolos en la implementación de soluciones de IA seguras y confiables.

## Oportunidades de Carrera
Anthropic ofrece un entorno propicio para el crecimiento profesional. Valoramos la diversidad de experiencias y la inclusión, y estamos buscando personas apasionadas que quieran ayudarnos a construir el futuro de la IA segura.

### Beneficios para Empleados
- **Salud y Bienestar**: Seguro médico, dental y visual, así como apoyo en salud mental.
- **Compensación**: Paquetes salariales competitivos con opciones de acciones.
- **Apoyo educativo**: Estipendios para educación y desarrollo profesional.

### Proceso de Entrevista
Nuestro proceso de selección está diseñado para conocer a candidatos que traigan habilidades únicas en un entorno multidisciplinario. No todos los roles requieren experiencia previa en aprendizaje automático, y todos son bienvenidos a postularse.

## Investigación
Conductamos investigaciones avanzadas en IA y exploramos áreas emergentes de seguridad, desde la interpretabilidad hasta el análisis de impactos sociales y políticos. Compartimos nuestras conclusiones y colaboramos con diversas partes interesadas para promover la seguridad en la industria.

## Contacto
¿Te gustaría unirte a nosotros o aprender más sobre nuestros productos y oportunidades? Visita nuestro sitio para más información.

[Visitar Anthropic](https://www.anthropic.com)

---

Este folleto proporciona una visión general de la empresa Anthropic, su cultura, productos y oportunidades laborales, destacando su compromiso con la seguridad y la confiabilidad en la inteligencia artificial.

In [20]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Página de Instructores', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Página de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Página para Empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Página de Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Página de Cursos', 'url': 'https://cursos.frogamesformacion.com/collections'}, {'type': 'Página de Carreras', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}, {'type': 'Página de Cursos de Matemáticas', 'url': 'https://cursos.frogamesformacion.com/bundles/aprende-matematicas-desde-cero'}]}


# Folleto Informativo de Frogames Formación

## **Acerca de Frogames**
Frogames Formación es una plataforma educativa galardonada, premiada como **la enseñanza online de datos y competencias digitales más innovadora de Europa en 2023**. Nuestro objetivo es ayudar a miles de estudiantes a convertirse en expertos en diversas áreas, como Programación de Videojuegos, Inteligencia Artificial, Machine Learning, y más.

---

## **Nuestra Cultura**
En Frogames, fomentamos un ambiente de aprendizaje divertido y emocionante. Creemos que aprender debe ser un viaje placentero acompañado de un fuerte sentido de comunidad. Nuestros instructores son profesionales dedicados que no solo comparten conocimientos, sino también una pasión por enseñar. La satisfacción de nuestros estudiantes es nuestra mayor recompensa, con más de **500.000 estudiantes satisfechos** en todo el mundo.

---

## **Clientes**
Nuestros estudiantes abarcan desde principiantes hasta profesionales en activo, interesados en mejorar sus competencias técnicas y digitales. Con una variedad de cursos y rutas temáticas, atendemos a individuos y empresas que buscan innovar y formar a su personal.  

### Testimonios de Estudiantes
- **Edwyn Mendoza**: "Los cursos son especializados y realmente prácticos."
- **Javiera Constanza Vallejos Jara**: "Me a gustado ser parte de una comunidad donde se aprende, crece y se disfruta."
  
---

## **Carreras/Empleos**
Frogames ofrece oportunidades de carrera no solo para estudiantes, sino también para profesionales que desean formar parte de un equipo innovador. Estamos en búsqueda constante de **instructores expertos** y colaboradores apasionados por la enseñanza digital. Además, contamos con un programa de afiliados que permite a personas con una audiencia ganar recompensas al promover nuestros cursos.

---

## **Cursos y Certificaciones**
En Frogames, ofrecemos cursos en áreas como:
- **Programación** (C#, Python, Unity, Unreal Engine)
- **Inteligencia Artificial y Machine Learning**
- **Data Science y Business Intelligence**
  
Nuestros cursos están estructurados en **rutas temáticas** que permiten a los estudiantes avanzar de manera progresiva. Además, al finalizar cada curso, se otorgan **certificaciones validas por Blockchain**, que garantizan la autenticidad y son perfectas para agregar a tu CV y perfil de LinkedIn.

### Cursos Destacados:
- *Curso completo de Python de la A a la Z*
- *Fórmula Secreta de las Bandas Sonoras de Videojuegos*
- *Machine Learning de la A a la Z*

---

## **Frogames para Empresas**
Proporcionamos un servicio especial para empresas que buscan formar a su personal. Ofrecemos paquetes de formación a medida, adaptados a las necesidades específicas de cada negocio. Las empresas pueden beneficiarse de acceso a un amplio catálogo de cursos y descuentos especiales.

---

## **Contáctanos**
Para más información sobre nuestros cursos, oportunidades de empleo o formación para empresas, no dudes en **contactarnos**. ¡Te esperamos en Frogames, donde la formación y el aprendizaje se convierten en aventura!

---

> "La educación no cambia el mundo, cambia a las personas que van a cambiar el mundo". - *Paulo Freire*.

---

**Frogames - Tu futuro, nuestra pasión.**

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de Pricing', 'url': 'https://www.anthropic.com/pricing'}]}


# Folleto de Anthropic

## Acerca de Anthropic

Anthropic es una empresa de investigación y seguridad en inteligencia artificial con sede en San Francisco. Nos dedicamos a construir sistemas de IA confiables, interpretables y manejables, llevando la seguridad de la IA a la vanguardia de nuestro trabajo. Creemos que la IA tendrá un impacto inmenso en el mundo y trabajamos para garantizar que las tecnologías avancen de manera responsable y beneficiosa.

## Nuestros Productos

### Claude
Conoce a **Claude**, nuestra familia de modelos de IA que incluye Claude 3.5 Sonnet y Claude 3.5 Haiku. Estos modelos están diseñados para realizar tareas complejas, analizar imágenes, generar código y procesar múltiples idiomas, ayudando a los usuarios a trabajar de manera más eficiente.

### API de Claude
Ofrecemos acceso a la API de Claude para que las empresas puedan integrar estas capacidades avanzadas de IA en sus aplicaciones, mejorando así la experiencia del cliente y optimizando flujos de trabajo.

## Cultura de la Empresa

### Valores de Anthropic
1. **Compromiso con la Misión**: Estamos aquí para garantizar que la IA transformadora beneficie a la humanidad.
2. **Alta Confianza**: Fomentamos un ambiente de trabajo basado en el respeto, la honestidad, y la colaboración.
3. **Colaboración**: Creemos en trabajar como un solo equipo hacia nuestra misión compartida.
4. **Enfoque Empírico**: Valoramos las soluciones simples y prácticas, aprendiendo continuamente de nuestras experiencias.

## Oportunidades Laborales

### Carrera en Anthropic
Anthropic busca individuos apasionados y diversos en diversos campos, desde la investigación y la ingeniería hasta la política pública y las operaciones. No es necesario tener experiencia previa en IA para aplicar a muchos de nuestros roles.

### Beneficios
- **Salud y Bienestar**: Seguros de salud, dental y de visión.
- **Equidad y Salario Competitivo**: Ofrecemos paquetes de compensación competitiva y bonos de equidad.
- **Fomento del Desarrollo Profesional**: Estipendios para educación y apoyo para la reubicación.

## Compromiso con la Seguridad en IA

En Anthropic, tratamos la seguridad de la IA como una ciencia sistemática. Realizamos investigación en la vanguardia de la IA, aplicamos técnicas de seguridad avanzadas, y colaboramos con diversas organizaciones para promover la seguridad en la industria.

### Investigación
Nuestras investigaciones abarcan desde la interpretación de modelos hasta el análisis de impactos sociales y políticos, buscando siempre mejorar la fiabilidad y la seguridad de nuestros sistemas de IA.

## Contacto y Más Información

### Hablemos
Si desea saber más sobre cómo podemos ayudarlos a elevar su negocio con la IA, o si está interesado en unirse a nosotros en nuestra misión de seguridad en IA, no dude en:
- Visitar nuestra [página web](https://www.anthropic.com)
- Seguirnos en [Twitter](https://twitter.com/anthropic) y [LinkedIn](https://linkedin.com/company/anthropic)

¡Juntos podemos construir un futuro donde la IA beneficie a todos!

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.